In [494]:
import math
import matplotlib.pyplot as plt

In [541]:
class Element_Pricing_Model_Paul:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def calc_in_given_out(out,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = pow(k-pow(out_reserves-out,1-t),1/(1-t)) - in_reserves
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            unit_price = out/without_fee
            fee =  (1-unit_price)*g
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*out
            with_fee = out * (1/(unit_price + fee))
            total_fee=without_fee-with_fee
            #print("unit price=amount of fyt/total_price_without_fee: " + str(unit_price) + "=" + str(out) + "/" + str(without_fee))
        else:
            unit_price = without_fee/out
            fee =  (1-unit_price)*g
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*out
            with_fee = out * (unit_price - fee)
            total_fee=out*fee
            #print("unit price=total_price_without_fee/amount of base: " + str(unit_price) + "=" + str(without_fee) + "/" + str(out))
        return (without_fee_or_slippage,with_fee,without_fee,total_fee)
    
    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def calc_out_given_in(in_,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = in_reserves - pow(k-pow(out_reserves+in_,1-t),1/(1-t))
        if in_reserves > out_reserves:
            # this condition is true when in is the fyt
            unit_price = in_/without_fee
            fee =  (1-unit_price)*g
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*in_
            with_fee = in_ * (1/(unit_price + fee))
            total_fee=without_fee-with_fee
            #print("unit price=amount of fyt/total_price_without_fee: " + str(unit_price) + "=" + str(in_) + "/" + str(without_fee))
        else:
            unit_price = without_fee/in_
            fee =  (1-unit_price)*g
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*in_
            with_fee = in_ * (unit_price - fee)
            total_fee=in_*fee
            #print("unit price=total_price_without_fee/amount of base: " + str(unit_price) + "=" + str(without_fee) + "/" + str(in_))
        
        return (without_fee_or_slippage,with_fee,without_fee,total_fee)


    
### Yield Space Method
# x^(1-t) + y^(1-t) = k
# -> y = (k - x^(1-t))^1/(1-t)

# (x-i)^(1-t) + (y+j)^(1-t) = k
# Sell j fyt for i base
# i = x - (k - (y+j)^(1-tg))^1/(1-tg)
# Buy i base for j fyt
# j = (k - (x-i)^(1-t/g))^1/(1-t/g) - y

class Yield_Pricing_Model:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def base_to_fyt(amount,x,y,g,t):
        k=pow(x,1-t/g) + pow(y,1-t/g)
        # calculate the number of fyts the specified amount of base tokens will cost with fees
        output_with_fee = pow(k-pow(x-amount,1-t/g),1/(1-t/g)) - y
        k=pow(x,1-t) + pow(y,1-t)
        # calculate the number of fyts the specified amount of base tokens will cost without fees
        output_without_fee = pow(k-pow(x-amount,1-t),1/(1-t)) - y
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        return (output_with_fee,output_without_fee,fee)

    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def fyt_to_base(amount,x,y,g,t):
        k=pow(x,1-t*g) + pow(y,1-t*g)
        # calculate the number of base tokens the specified amount of fy tokens will cost with fees
        output_with_fee = x - pow(k-pow(y+amount,1-t*g),1/(1-t*g))
        k=pow(x,1-t) + pow(y,1-t)
        # calculate the number of base tokens the specified amount of fy tokens will cost without fees
        output_without_fee = x - pow(k-pow(y+amount,1-t),1/(1-t))
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        return (output_with_fee,output_without_fee,fee)
    

class Market: 
    def __init__(self,x,y,g,t,pricing_model): 
        self.x=x
        self.y=y
        self.total_supply = x + y
        self.g=g
        self.t=t
        self.pricing_model=pricing_model
        self.x_orders = 0
        self.y_orders = 0
        self.x_volume = 0
        self.y_volume = 0
        self.cum_y_slippage=0
        self.cum_x_slippage=0
        self.cum_y_fees=0
        self.cum_x_fees=0
        self.starting_fyt_price=self.fyt_price()
    
    def apy(self):
        return ((self.y+self.total_supply)/self.x - 1) * 100
    
    def fyt_price(self):
        return 1/pow((self.y+self.total_supply)/self.x,self.t)
    
    def tick(self,step_size):
        self.t -= step_size
    
    def swap(self, amount, token_in, token_out):
        if token_in == "base" and token_out == "fyt":
            (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.buy_base_with_fyt(amount)
            self.y_orders+=1
            self.y_volume+=output_with_fee
        elif token_in == "fyt" and token_out == "base":
            (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.buy_fyt_with_base(amount)
            self.x_orders+=1
            self.x_volume+=output_with_fee
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)
        
    # input: amount of base token to buy
    def buy_base_with_fyt(self,amount):
        (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_in_given_out(amount,self.y+self.total_supply,self.x,self.g,self.t)
        self.x += amount
        self.y -= output_with_fee
        self.cum_y_slippage += abs(without_fee_or_slippage-output_without_fee)
        self.cum_y_fees += fee
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)
    
    # input: amount of fy token to buy
    def buy_fyt_with_base(self,amount):
        (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_out_given_in(amount,self.x,self.y+self.total_supply,self.g,self.t)
        self.x -= output_with_fee
        self.y += amount
        self.cum_x_slippage += abs(without_fee_or_slippage-output_without_fee)
        self.cum_x_fees += fee
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)


In [540]:
display(Element_Pricing_Model_Paul.calc_in_given_out(100,5000+55000,50000,.05,.9))
display(Element_Pricing_Model_Paul.calc_in_given_out(100,50000,5000+55000,.05,.9))


display(Element_Pricing_Model_Paul.calc_out_given_in(100,50000,5000+55000,.05,.9))
display(Element_Pricing_Model_Paul.calc_out_given_in(100,5000+55000,50000,.05,.9))

(117.83196534742953,
 116.98715214804606,
 118.04252389188332,
 1.0553717438372558)

(84.86661467891868, 84.24504376165787, 84.9952797730075, 0.7502360113496254)

(84.86661467891868, 83.9752539549958, 84.73833709999599, 0.7630831450002007)

(117.83196534742953,
 116.59483304153582,
 117.62215940857277,
 1.0273263670369488)

In [543]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000.0
y_start = 5000.0
g=.05
t=.9
max_order_size=10
step_size=.0005
m = Market(x_start,y_start,g,t,Element_Pricing_Model_Paul)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
trades = []
init = {
    "percent_fee": g,
    "initial_x_reserves": m.x,
    "initial_y_reserves": m.y,
    "initial_apy": m.apy(),
    "initial_fyt_price": m.fyt_price(),
    "max_order_size": max_order_size,
    "total_supply": m.total_supply,
}
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,max_order_size)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    trade_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (without_fee_or_slippage,with_fee,without_fee,fee) = m.swap(amount,token_in,token_out)
    trade_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": with_fee,
        "fee": fee
    }
    
    trades.append({
        "input": trade_input,
        "output": trade_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum slippage x: " + str(m.cum_x_slippage))
print("Cum slippage y: " + str(m.cum_y_slippage))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending Reserve Ratio: " + str(m.apy()/100))
print("Ending Slippage Ratio: " + str(m.cum_y_slippage/m.cum_x_slippage - 1))
print("Ending Fee Ratio: " + str(m.cum_y_fees/m.cum_x_fees - 1))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

run={
    "init":init,
    "trades":trades
}

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000.0
Starting Y Reserves: 5000.0
Ending X Reserves: 50018.922881584534
Ending Y Reserves: 5013.518740557928
Num x orders: 916
Cum x volume: 4266.079247258259
Num y orders: 884
Cum y volume: 4633.238411325852
Cum slippage x: 0.2165185629933417
Cum slippage y: 0.27446389717220165
Cum fees x: 18.127519267881585
Cum fees y: 20.46064234943259
Ending Reserve Ratio: 0.19981629517762164
Ending Slippage Ratio: 0.2676229390116627
Ending Fee Ratio: 0.12870614269238945
Ending FYT Price: 0.999999999999992
Ending Time: 4.3635234314720606e-14


In [536]:
#%pip install genson
import json
from genson import SchemaBuilder

with open('test_vectors.json', 'w') as fp:
    json.dump(run, fp, indent=1)
    
builder = SchemaBuilder()
builder.add_object(run)
run_schema=builder.to_schema()

with open('test_vectors_schema.json', 'w') as fp:
    json.dump(run_schema, fp, indent=1)

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000
Starting Y Reserves: 5000
Ending X Reserves: 49978.22767074605
Ending Y Reserves: 4988.36531082712
Num x orders: 884
Cum x volume: 4285.002128842804
Num y orders: 916
Cum y volume: 4646.757151883774
Cum slippage x: 0.2165356884307857
Cum slippage y: 0.27460085034122034
Cum fees x: 18.2632072521446
Cum fees y: 20.833606283734227
Ending Reserve Ratio: 0.20028996838437998
Ending Slippage Ratio: 0.2681551587695661
Ending Fee Ratio: 0.14074192972254607
Ending FYT Price: 0.999999999999992
Ending Time: 4.3635234314720606e-14


In [544]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000.0
y_start = 5000.0
g=.95
t=.9
max_order_size=10
step_size=.0005
m = Market(x_start,y_start,g,t,Yield_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
trades = []
init = {
    "percent_fee": g,
    "initial_x_reserves": m.x,
    "initial_y_reserves": m.y,
    "initial_apy": m.apy(),
    "initial_fyt_price": m.fyt_price(),
    "max_order_size": max_order_size,
    "total_supply": m.total_supply,
}
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,max_order_size)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    trade_input = {
        "time": m.t,
        "x_reserves": m.x,
        "y_reserves": m.y,
        "token_in": token_in,
        "amount_in": amount,
        "token_out": token_out
    }
    (without_fee_or_slippage,with_fee,without_fee,fee) = m.swap(amount,token_in,token_out)
    trade_output = {
        "x_reserves": m.x,
        "y_reserves": m.y,
        "amount_out": with_fee,
        "fee": fee
    }
    
    trades.append({
        "input": trade_input,
        "output": trade_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum slippage x: " + str(m.cum_x_slippage))
print("Cum slippage y: " + str(m.cum_y_slippage))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending Reserve Ratio: " + str(m.apy()/100))
print("Ending Slippage Ratio: " + str(m.cum_y_slippage/m.cum_x_slippage - 1))
print("Ending Fee Ratio: " + str(m.cum_y_fees/m.cum_x_fees - 1))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

run={
    "init":init,
    "trades":trades
}

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000.0
Starting Y Reserves: 5000.0


AttributeError: type object 'Yield_Pricing_Model' has no attribute 'calc_out_given_in'